In [ ]:
# -*- coding: utf-8 -*-
import os
import json
import time
import csv
import gc
import logging
from typing import Optional, Tuple

import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from totalsegmentator.python_api import totalsegmentator

# ========================= Logging =========================
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# =================== Helper: Zeitformat & CSV-Summe ===================
def _fmt(sec: float) -> str:
    h, r = divmod(sec, 3600)
    m, s = divmod(r, 60)
    return f"{int(h):02d}:{int(m):02d}:{s:05.2f}"

def _sum_seconds_from_csv(csv_path: str) -> float:
    """
    Summiert die Spalte 'seconds' aus timing_per_slice.csv über alle Runs.
    Robuste Auswertung ohne pandas.
    """
    if not os.path.exists(csv_path):
        return 0.0
    total = 0.0
    try:
        with open(csv_path, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            if not reader.fieldnames or "seconds" not in reader.fieldnames:
                return 0.0
            for row in reader:
                try:
                    total += float(row["seconds"])
                except Exception:
                    continue
    except Exception as e:
        logging.warning(f"Konnte CSV nicht auswerten ({csv_path}): {e}")
    return total

# =================== Laufzeit-Status (akkumuliert) ===================
def _load_runtime_state(state_path: str) -> dict:
    if os.path.exists(state_path):
        try:
            with open(state_path, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception as e:
            logging.warning(f"Konnte Runtime-State nicht lesen: {e}")
    return {"sessions": [], "total_elapsed_sec": 0.0}

def _save_runtime_state(state_path: str, state: dict):
    with open(state_path, "w", encoding="utf-8") as f:
        json.dump(state, f, indent=2)

def time_run_accumulate(func, state_path: str, seed_timing_csv: str, *args, **kwargs):
    """
    Führt func aus, misst Laufzeit des aktuellen Runs und akkumuliert in state_path.
    Zusätzlich wird am Ende die *echte* Gesamtlaufzeit über alle Neustarts
    aus timing_per_slice.csv (seed_timing_csv) berechnet und geloggt.
    """
    state = _load_runtime_state(state_path)
    start = time.perf_counter()
    start_wall = time.time()
    logging.info("==== Starte Laufzeitmessung (akkumuliert) ====")
    try:
        return func(*args, **kwargs)
    finally:
        elapsed = time.perf_counter() - start
        end_wall = time.time()
        state["sessions"].append({"start": start_wall, "end": end_wall, "elapsed_sec": elapsed})
        state["total_elapsed_sec"] = float(state.get("total_elapsed_sec", 0.0)) + float(elapsed)
        _save_runtime_state(state_path, state)

        msg1 = f"Aktueller Run: {_fmt(elapsed)} | Akkumuliert (Wall-Clock): {_fmt(state['total_elapsed_sec'])}"
        logging.info(msg1)

        total_csv_sec = _sum_seconds_from_csv(seed_timing_csv)
        msg2 = f"Akkumulierte Gesamtlaufzeit (aus CSV, alle Neustarts): {_fmt(total_csv_sec)}"
        logging.info(msg2)

        with open("runtime_log_upper_airways.txt", "a", encoding="utf-8") as f:
            f.write(msg1 + "\n")
            f.write(msg2 + "\n")

# =================== Resume-Checkpoint ===================
def save_resume_state(checkpoint_path: str, filename: str, slice_idx: int):
    state = {"resume_file": filename, "resume_slice": slice_idx}
    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(state, f)

def load_resume_state(checkpoint_path: str) -> Tuple[Optional[str], Optional[int]]:
    if os.path.exists(checkpoint_path):
        try:
            with open(checkpoint_path, "r", encoding="utf-8") as f:
                state = json.load(f)
            return state.get("resume_file"), state.get("resume_slice")
        except Exception as e:
            logging.warning(f"Checkpoint konnte nicht gelesen werden: {e}")
    return None, None

# =================== Eingabe-/Ausgabeordner ===================
images_dir = "data/upper_airways/filtered_images"
output_dir = "data/upper_airways/combined_segmentations_with_runtime"
os.makedirs(output_dir, exist_ok=True)

# Runtime/Resume Artefakte im selben Ordner
checkpoint_path = os.path.join(output_dir, "resume_checkpoint.json")
timing_csv      = os.path.join(output_dir, "timing_per_slice.csv")
runtime_state   = os.path.join(output_dir, "runtime_state.json")

# =================== Organe pro Task (UNVERÄNDERT) ===================
task_organs = {
    "head_glands_cavities": ["nasal_cavity_right", "nasal_cavity_left", "nasopharynx", "oropharynx", "hypopharynx"],
    "headneck_bones_vessels": ["larynx_air"],
    # "total": ["trachea"]
    "total": ["trachea", "lung_upper_lobe_left", "lung_upper_lobe_right"]
}

# =================== Visualisierung (wie bei dir) ===================
def _save_visualization(img_slice: np.ndarray, mask_slice: np.ndarray, save_path: str, slice_idx: int):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(img_slice, cmap="gray")
    ax[0].set_title(f"Original Slice {slice_idx}")
    ax[0].axis('off')
    ax[1].imshow(img_slice, cmap="gray")
    ax[1].imshow(mask_slice, cmap="jet", alpha=0.5)
    ax[1].set_title(f"Kombinierte Maske Slice {slice_idx}")
    ax[1].axis('off')
    plt.savefig(save_path, bbox_inches="tight")
    plt.close(fig)

# =================== Pipeline (deine Logik + Laufzeit/Resume) ===================
def segment_and_combine_with_runtime(
    images_dir: str,
    output_dir: str,
    checkpoint_path: str,
    timing_csv: str,
    use_auto_resume: bool = True,
    resume_file: Optional[str] = None,
    resume_slice: Optional[int] = None,
):
    image_files = sorted(os.listdir(images_dir))

    # Auto-Resume
    if use_auto_resume and (resume_file is None and resume_slice is None):
        cf, cs = load_resume_state(checkpoint_path)
        if cf is not None:
            logging.info(f"Automatischer Resume → Datei={cf}, Slice={cs}")
            resume_file, resume_slice = cf, cs

    # timing CSV initialisieren
    if not os.path.exists(timing_csv):
        with open(timing_csv, "w", newline="", encoding="utf-8") as fcsv:
            csv.writer(fcsv).writerow(["file", "slice_idx", "seconds"])

    temp_output_base = os.path.join(output_dir, "temp_segmentations_upper_airway")
    os.makedirs(temp_output_base, exist_ok=True)

    resume_armed = resume_file is None  # erst ab resume_file loslegen

    for image_file in tqdm(image_files, desc="Processing"):
        if not (image_file.endswith(".nii") or image_file.endswith(".nii.gz")):
            continue

        if not resume_armed:
            if image_file == resume_file:
                resume_armed = True
            else:
                continue

        image_path = os.path.join(images_dir, image_file)
        base_name = os.path.splitext(os.path.splitext(image_file)[0])[0]
        temp_output = os.path.join(temp_output_base, base_name)
        os.makedirs(temp_output, exist_ok=True)

        try:
            nii_img = nib.load(image_path)
            image_data = nii_img.get_fdata()
        except Exception as e:
            logging.error(f"Fehler beim Laden von {image_file}: {e}")
            continue

        total_slices = image_data.shape[2]
        combined_mask = np.zeros_like(image_data, dtype=np.uint8)

        # ---- (UNVERÄNDERT) Segmentieren pro Task & Organe kombinieren
        for task, organs in task_organs.items():
            try:
                totalsegmentator(input=image_path, output=temp_output, task=task)
            except Exception as e:
                logging.error(f"Fehler bei Task {task} für {image_file}: {e}")
                continue

            for organ in organs:
                seg_path = os.path.join(temp_output, f"{organ}.nii.gz")
                if not os.path.exists(seg_path):
                    logging.warning(f"{organ} fehlt in {image_file} (Task: {task})")
                    continue

                try:
                    seg_data = nib.load(seg_path).get_fdata()
                    combined_mask += (seg_data > 0).astype(np.uint8)
                except Exception as e:
                    logging.error(f"Fehler beim Laden von {organ} für {image_file}: {e}")

        # ---- Slice-weise Verarbeitung + Laufzeit-Messung + Resume
        slice_start = resume_slice if (image_file == resume_file and resume_slice is not None) else 0
        original_size = None
        target_size = None

        for slice_idx in range(slice_start, total_slices):
            t0 = time.perf_counter()
            try:
                # Checkpoint VOR der Bearbeitung des Slices schreiben
                save_resume_state(checkpoint_path, image_file, slice_idx)

                img_slice = image_data[:, :, slice_idx]
                mask_slice = combined_mask[:, :, slice_idx]

                # Zielgröße nur bei Bedarf setzen
                if slice_idx == 0 or original_size is None:
                    original_size = (mask_slice.shape[1], mask_slice.shape[0])  # (width, height)
                    target_size = (1024, 1024) if original_size != (1024, 1024) else original_size

                # Visualisierung
                vis_path = os.path.join(output_dir, f"{base_name}_slice_{slice_idx}_combined.png")
                _save_visualization(img_slice, mask_slice, vis_path, slice_idx)

                # Maske skalieren nur wenn nötig
                if original_size != target_size:
                    mask_resized = cv2.resize(mask_slice.astype(np.uint8), target_size, interpolation=cv2.INTER_NEAREST)
                else:
                    mask_resized = mask_slice.astype(np.uint8)

                # Maske speichern
                mask_path = os.path.join(output_dir, f"{base_name}_slice_{slice_idx}_combined_mask.png")
                cv2.imwrite(mask_path, mask_resized * 255)

                gc.collect()

            except Exception as e:
                logging.error(f"Fehler bei {image_file} Slice {slice_idx}: {e}")
            finally:
                elapsed_slice = time.perf_counter() - t0
                with open(timing_csv, "a", newline="", encoding="utf-8") as fcsv:
                    csv.writer(fcsv).writerow([image_file, slice_idx, f"{elapsed_slice:.4f}"])

    # Fertig → Checkpoint löschen
    if os.path.exists(checkpoint_path):
        os.remove(checkpoint_path)
        logging.info("Checkpoint entfernt (Pipeline abgeschlossen).")

# =================== AUSFÜHRUNG ===================
if __name__ == "__main__":
    # Optionales manuelles Resume (sonst Auto-Resume)
    manual_resume_file  = None  # z.B. "URT_001.nii.gz"
    manual_resume_slice = None  # z.B. 120

    # Lauf starten (Wall-Clock akkumuliert + CSV-basierte Gesamtzeit)
    time_run_accumulate(
        segment_and_combine_with_runtime,
        runtime_state,                   # state_path
        seed_timing_csv=timing_csv,      # für Summen-Log am Ende
        images_dir=images_dir,
        output_dir=output_dir,
        checkpoint_path=checkpoint_path,
        timing_csv=timing_csv,
        use_auto_resume=True,
        resume_file=manual_resume_file,
        resume_slice=manual_resume_slice,
    )

    # Finales Log der CSV-Gesamtlaufzeit
    total_csv_sec = _sum_seconds_from_csv(timing_csv)
    logging.info(f"[FINAL] Gesamtlaufzeit aus CSV (alle Neustarts): {_fmt(total_csv_sec)}")


2025-09-18 09:15:51,498 - INFO - ==== Starte Laufzeitmessung (akkumuliert) ====
Processing:   0%|                                                                               | 0/31 [00:00<?, ?it/s]

No GPU detected. Running on CPU. This can be very slow. The '--fast' or the `--roi_subset` option can help to reduce runtime.

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough segmentation for cropping...
Resampling...
  Resampled in 0.44s
Predicting...



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.53s/it]


  Predicted in 40.94s
Resampling...
  cropping from (512, 512, 32) to (407, 388, 32)
Resampling...
  Resampled in 2.26s
Predicting...



100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:30<00:00, 11.27s/it]


...

Predicting part 4 of 5 ...



100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:38<00:00, 12.29s/it]


Predicting part 5 of 5 ...



100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:38<00:00, 12.33s/it]


  Predicted in 590.71s
Resampling...
Saving segmentations...
  Saved in 43.79s


In [1]:
import pandas as pd

# Pfad zur timing_per_slice.csv
csv_path = "data//upper_airways/combined_segmentations_with_runtime/timing_per_slice.csv"

# CSV laden
df = pd.read_csv(csv_path)

# Spalte seconds sicher als float laden
df['seconds'] = pd.to_numeric(df['seconds'], errors='coerce')

# Gesamtzeit
total_seconds = df['seconds'].sum()
total_minutes = total_seconds / 60
total_hours = total_seconds / 3600

# Anzahl der Slices
num_slices = len(df)

# Durchschnittszeit
avg_sec = df['seconds'].mean()
avg_min = avg_sec / 60

# Min/Max/Median
min_sec = df['seconds'].min()
max_sec = df['seconds'].max()
median_sec = df['seconds'].median()

# Ausgabe auf der Konsole
print("==== Timing Summary ====")
print(f"Slices verarbeitet : {num_slices}")
print(f"Gesamtzeit        : {total_seconds:.2f} Sekunden")
print(f"                   : {total_minutes:.2f} Minuten")
print(f"                   : {total_hours:.2f} Stunden")
print(f"Durchschnitt/Slice: {avg_sec:.2f} Sekunden ({avg_min:.2f} Minuten)")
print(f"Median/Slice      : {median_sec:.2f} Sekunden")
print(f"Schnellster Slice : {min_sec:.2f} Sekunden")
print(f"Langsamster Slice : {max_sec:.2f} Sekunden")

# Optional: Zusammenfassung in eine Textdatei speichern
summary_path = "data//upper_airways/combined_segmentations_with_runtime/timing_summary.txt"
with open(summary_path, "w") as f:
    f.write("==== Timing Summary ====\n")
    f.write(f"Slices verarbeitet : {num_slices}\n")
    f.write(f"Gesamtzeit        : {total_seconds:.2f} Sekunden\n")
    f.write(f"                   : {total_minutes:.2f} Minuten\n")
    f.write(f"                   : {total_hours:.2f} Stunden\n")
    f.write(f"Durchschnitt/Slice: {avg_sec:.2f} Sekunden ({avg_min:.2f} Minuten)\n")
    f.write(f"Median/Slice      : {median_sec:.2f} Sekunden\n")
    f.write(f"Schnellster Slice : {min_sec:.2f} Sekunden\n")
    f.write(f"Langsamster Slice : {max_sec:.2f} Sekunden\n")

print(f"\nZusammenfassung gespeichert in {summary_path}")


==== Timing Summary ====
Slices verarbeitet : 5541
Gesamtzeit        : 5293.67 Sekunden
                   : 88.23 Minuten
                   : 1.47 Stunden
Durchschnitt/Slice: 0.96 Sekunden (0.02 Minuten)
Median/Slice      : 0.75 Sekunden
Schnellster Slice : 0.66 Sekunden
Langsamster Slice : 8.44 Sekunden

Zusammenfassung gespeichert in data//upper_airways/combined_segmentations_with_runtime/timing_summary.txt


In [2]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import gc
import logging
from tqdm import tqdm
from totalsegmentator.python_api import totalsegmentator

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Eingabe- und Ausgabeordner
images_dir = "data/upper_airways/filtered_images"
output_dir = "data/upper_airways/combined_segmentations"
os.makedirs(output_dir, exist_ok=True)

# Organe pro Task
task_organs = {
    "head_glands_cavities": ["nasal_cavity_right", "nasal_cavity_left", "nasopharynx", "oropharynx", "hypopharynx"],
    "headneck_bones_vessels": ["larynx_air"],
    #"total": ["trachea"]
    "total": ["trachea", "lung_upper_lobe_left", "lung_upper_lobe_right"]
    
}

def segment_and_combine(images_dir, output_dir):
    image_files = sorted(os.listdir(images_dir))
    temp_output_base = os.path.join(output_dir, "temp_segmentations_upper_airway")
    os.makedirs(temp_output_base, exist_ok=True)

    for image_file in tqdm(image_files, desc="Processing"):
        if not (image_file.endswith(".nii") or image_file.endswith(".nii.gz")):
            continue

        image_path = os.path.join(images_dir, image_file)
        base_name = os.path.splitext(os.path.splitext(image_file)[0])[0]
        temp_output = os.path.join(temp_output_base, base_name)
        os.makedirs(temp_output, exist_ok=True)

        try:
            nii_img = nib.load(image_path)
            image_data = nii_img.get_fdata()
        except Exception as e:
            logging.error(f"Fehler beim Laden von {image_file}: {e}")
            continue

        total_slices = image_data.shape[2]
        combined_mask = np.zeros_like(image_data, dtype=np.uint8)

        for task, organs in task_organs.items():
            try:
                totalsegmentator(input=image_path, output=temp_output, task=task)
            except Exception as e:
                logging.error(f"Fehler bei Task {task} für {image_file}: {e}")
                continue

            for organ in organs:
                seg_path = os.path.join(temp_output, f"{organ}.nii.gz")
                if not os.path.exists(seg_path):
                    logging.warning(f"{organ} fehlt in {image_file} (Task: {task})")
                    continue

                try:
                    seg_data = nib.load(seg_path).get_fdata()
                    combined_mask += (seg_data > 0).astype(np.uint8)
                except Exception as e:
                    logging.error(f"Fehler beim Laden von {organ} für {image_file}: {e}")

        # Slice-weise Verarbeitung
        for slice_idx in range(total_slices):
            img_slice = image_data[:, :, slice_idx]
            mask_slice = combined_mask[:, :, slice_idx]

            # Zielgröße nur bei Bedarf setzen
            if slice_idx == 0:
                original_size = (mask_slice.shape[1], mask_slice.shape[0])  # (width, height)
                target_size = (1024, 1024) if original_size != (1024, 1024) else original_size

            # Visualisierung
            fig, ax = plt.subplots(1, 2, figsize=(12, 6))
            ax[0].imshow(img_slice, cmap="gray")
            ax[0].set_title(f"Original Slice {slice_idx}")
            ax[0].axis('off')
            ax[1].imshow(img_slice, cmap="gray")
            ax[1].imshow(mask_slice, cmap="jet", alpha=0.5)
            ax[1].set_title(f"Kombinierte Maske Slice {slice_idx}")
            ax[1].axis('off')

            vis_path = os.path.join(output_dir, f"{base_name}_slice_{slice_idx}_combined.png")
            plt.savefig(vis_path, bbox_inches="tight")
            plt.close(fig)

            # Maske skalieren nur wenn nötig
            if original_size != target_size:
                mask_resized = cv2.resize(mask_slice.astype(np.uint8), target_size, interpolation=cv2.INTER_NEAREST)
            else:
                mask_resized = mask_slice.astype(np.uint8)

            # Maske speichern
            mask_path = os.path.join(output_dir, f"{base_name}_slice_{slice_idx}_combined_mask.png")
            cv2.imwrite(mask_path, mask_resized * 255)

            gc.collect()

# Funktion ausführen
segment_and_combine(images_dir, output_dir)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.90s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:55<00:00, 14.48s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.88s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:26<00:00, 26.52s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:55<00:00, 13.96s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:58<00:00, 14.72s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:54<00:00, 13.62s/it]

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:59<00:00, 14.83s/it]

100%|███████████████████████████

## Dice Score Berechnung und CSV-Speicherung

In [3]:
import os
import numpy as np
import cv2
import logging
import csv
from concurrent.futures import ThreadPoolExecutor

# Konfiguration
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
ground_truth_folder = "data/upper_airways/filtered_masks_png"
prediction_folder = "data/upper_airways/combined_segmentations"
output_csv = os.path.join(prediction_folder, "dice_scores_combined_segmentations.csv")

# Dice Score Funktion
def dice_score(mask1, mask2):
    intersection = np.sum(mask1 * mask2)
    if np.sum(mask1) + np.sum(mask2) == 0:
        return 1.0
    return (2. * intersection) / (np.sum(mask1) + np.sum(mask2))

# Eine Datei vergleichen
def process_single_file(gt_filename):
    try:
        base = gt_filename.replace("_Grund_truth_mask.png", "")
        pred_filename = f"{base}_combined_mask.png"

        gt_path = os.path.join(ground_truth_folder, gt_filename)
        pred_path = os.path.join(prediction_folder, pred_filename)

        if not os.path.exists(pred_path):
            logging.warning(f"Fehlende Vorhersagemaske: {pred_filename}")
            return None

        gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
        pred = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)

        if gt is None or pred is None:
            logging.error(f"Fehler beim Laden: {gt_filename}")
            return None

        gt = (gt > 0).astype(np.uint8)
        pred = (pred > 0).astype(np.uint8)

        score = dice_score(gt, pred)
        logging.info(f"{gt_filename}: Dice Score = {score:.4f}")
        return gt_filename, score
    except Exception as e:
        logging.error(f"Fehler bei {gt_filename}: {e}")
        return None

# Verarbeitung starten
dice_scores = []
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(process_single_file, fname)
        for fname in os.listdir(ground_truth_folder)
        if fname.endswith("_Grund_truth_mask.png")
    ]
    dice_scores = [f.result() for f in futures if f.result() is not None]

# CSV speichern
if dice_scores:
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Dateiname", "Dice Score"])
        writer.writerows(dice_scores)
    logging.info(f"Erfolgreich gespeichert: {output_csv}")
else:
    logging.info("Keine Dice Scores berechnet.")


2025-08-24 10:56:12,834 - INFO - atm_001_slice_5_Grund_truth_mask.png: Dice Score = 0.6293
2025-08-24 10:56:12,857 - INFO - atm_001_slice_6_Grund_truth_mask.png: Dice Score = 0.8489
2025-08-24 10:56:12,859 - INFO - atm_001_slice_4_Grund_truth_mask.png: Dice Score = 0.0000
2025-08-24 10:56:12,893 - INFO - atm_001_slice_7_Grund_truth_mask.png: Dice Score = 0.3178
2025-08-24 10:56:12,886 - INFO - atm_001_slice_1_Grund_truth_mask.png: Dice Score = 0.0000
2025-08-24 10:56:12,888 - INFO - atm_001_slice_2_Grund_truth_mask.png: Dice Score = 0.0000
2025-08-24 10:56:12,892 - INFO - atm_001_slice_0_Grund_truth_mask.png: Dice Score = 0.0000
2025-08-24 10:56:12,870 - INFO - atm_001_slice_3_Grund_truth_mask.png: Dice Score = 0.0000
2025-08-24 10:56:13,011 - INFO - atm_001_slice_9_Grund_truth_mask.png: Dice Score = 0.6177
2025-08-24 10:56:13,017 - INFO - atm_001_slice_11_Grund_truth_mask.png: Dice Score = 0.2219
2025-08-24 10:56:13,017 - INFO - atm_001_slice_10_Grund_truth_mask.png: Dice Score = 0.40

## Histogramm aus CSV-Datei generieren

In [6]:
import os
import matplotlib.pyplot as plt
import csv
import logging

# Konfiguration
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
csv_path = "data/upper_airways/combined_segmentations/dice_scores_combined_segmentations.csv"
output_plot = "data/upper_airways/dice_score_histogram_Totalsegmentator_upperairways.png"

# Dice-Werte laden
dice_values = []
try:
    with open(csv_path, newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                dice_values.append(float(row["Dice Score"]))
            except ValueError:
                logging.warning(f"Ungültiger Wert: {row['Dice Score']}")
except FileNotFoundError:
    logging.error(f"CSV-Datei nicht gefunden: {csv_path}")
    dice_values = []

# Histogramm erzeugen
if dice_values:
    avg_dice = sum(dice_values) / len(dice_values)

    plt.figure(figsize=(8, 6))
    plt.hist(dice_values, bins=20, color="blue", alpha=0.7, edgecolor="black")
    plt.axvline(avg_dice, color="red", linestyle="--", label=f"Mean: {avg_dice:.4f}")
    plt.title("Dice Score Verteilung")
    plt.xlabel("Dice Score")
    plt.ylabel("Häufigkeit")
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    #plt.grid(True)

    plt.savefig(output_plot)
    plt.show()
    logging.info(f"Histogramm gespeichert: {output_plot}")
else:
    logging.info("Keine Dice-Werte zum Plotten gefunden.")


C:\Users\rawan\AppData\Local\Temp\ipykernel_10976\2468796639.py:41: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
2025-08-24 11:14:00,651 - INFO - Histogramm gespeichert: data/upper_airways/dice_score_histogram_Totalsegmentator_upperairways.png


In [9]:
import os
import csv
import cv2
import glob
import re
import numpy as np
import logging
import matplotlib
matplotlib.use("Agg")  # sicher speichern ohne Display
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor

# =========================
# Konfiguration
# =========================
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

ground_truth_folder = "data/upper_airways/filtered_masks_png"
prediction_folder   = "data/upper_airways/combined_segmentations"

output_csv          = os.path.join(prediction_folder, "dice_scores_combined_segmentations_no_zeros.csv")
output_plot_main    = "data/upper_airways/dice_score_histogram_Totalsegmentator_upperairways_no_zeros.png"
output_plot_fallback= "data/upper_airways/dice_score_histogram_Totalsegmentator_upperairways_with_zeros.png"

# Verhalten steuern
SKIP_EMPTY_GT_SLICES  = True     # Slices mit leerer GT überspringen
EXCLUDE_ZERO_IN_STATS = True     # 0-Werte in Statistik/Histogramm ausschließen
SHOW_PLOT             = False    # nur speichern

# GT-Dateimuster
GT_SUFFIX = "_Grund_truth_mask.png"   # prüfe, ob das exakt passt!

# Mögliche Pred-Suffixe (erweitern, falls nötig)
PRED_PATTERNS = [
    "{base}_combined_mask.png",
    "{base}_segmentation.png",
    "{base}_segmentation_TotalSegmentator.png",
    "{base}_combined_segmentation.png",
    "{base}_pred.png",
    "{base}.png",
]

# =========================
# Utils
# =========================
def dice_score(mask1: np.ndarray, mask2: np.ndarray) -> float:
    intersection = np.sum(mask1 * mask2, dtype=np.uint64)
    denom = np.sum(mask1, dtype=np.uint64) + np.sum(mask2, dtype=np.uint64)
    if denom == 0:
        return 1.0
    return (2.0 * intersection) / denom

def find_prediction_path(base: str) -> str | None:
    """
    Versuche zuerst bekannte Muster. Fallback: glob nach base*.png im Prediction-Ordner.
    """
    # 1) bekannte Muster durchprobieren
    for pat in PRED_PATTERNS:
        cand = os.path.join(prediction_folder, pat.format(base=base))
        if os.path.exists(cand):
            return cand

    # 2) Fallback: nimm die erste PNG, die mit base beginnt
    candidates = sorted(glob.glob(os.path.join(prediction_folder, base + "*.png")))
    if candidates:
        return candidates[0]

    return None

def save_histogram(values, title, xlabel, ylabel, mean_label, out_path):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    plt.figure(figsize=(8, 6))
    plt.hist(values, bins=20, alpha=0.7, edgecolor="black")
    plt.axvline(np.mean(values), linestyle="--", label=f"{mean_label}: {np.mean(values):.4f}")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    if SHOW_PLOT:
        plt.show()
    plt.close()
    logging.info(f"Histogramm gespeichert: {out_path}")

# =========================
# Einzeldatei verarbeiten
# =========================
def process_single_file(gt_filename: str):
    try:
        if not gt_filename.endswith(GT_SUFFIX):
            return None, "skip_pattern"

        base = gt_filename[:-len(GT_SUFFIX)]  # sicherer als replace()
        gt_path = os.path.join(ground_truth_folder, gt_filename)
        pred_path = find_prediction_path(base)

        if pred_path is None:
            return None, "missing_pred"

        gt   = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
        pred = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)

        if gt is None or pred is None:
            return None, "load_error"

        gt_bin   = (gt > 0).astype(np.uint8)
        pred_bin = (pred > 0).astype(np.uint8)

        if SKIP_EMPTY_GT_SLICES and gt_bin.sum() == 0:
            return None, "empty_gt_skipped"

        score = float(dice_score(gt_bin, pred_bin))
        return (gt_filename, score), "ok"

    except Exception as e:
        logging.error(f"Fehler bei {gt_filename}: {e}")
        return None, "exception"

# =========================
# Hauptlauf
# =========================
def main():
    # Quick Sanity-Logs
    gt_files = sorted([f for f in os.listdir(ground_truth_folder) if f.endswith(".png")])
    pred_files = sorted([f for f in os.listdir(prediction_folder) if f.endswith(".png")])

    logging.info(f"GT-Dateien gefunden: {len(gt_files)}")
    logging.info(f"Pred-Dateien gefunden: {len(pred_files)}")

    if gt_files[:5]:
        logging.info("Beispiel GT-Dateien: " + ", ".join(gt_files[:5]))
    if pred_files[:5]:
        logging.info("Beispiel Pred-Dateien: " + ", ".join(pred_files[:5]))

    # Nur jene GT, die unser Suffix tragen
    gt_files = [f for f in gt_files if f.endswith(GT_SUFFIX)]
    if not gt_files:
        logging.error(f"Keine GT-Dateien mit Suffix '{GT_SUFFIX}' gefunden.")
        return

    status_counts = {"ok":0, "missing_pred":0, "load_error":0, "empty_gt_skipped":0, "exception":0, "skip_pattern":0}

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_single_file, fname) for fname in gt_files]
        results = [f.result() for f in futures]

    dice_items = []
    for item, status in results:
        status_counts[status] = status_counts.get(status, 0) + 1
        if status == "ok" and item is not None:
            dice_items.append(item)

    logging.info("Verarbeitet: " + ", ".join([f"{k}={v}" for k, v in status_counts.items()]))

    if not dice_items:
        logging.error("Keine Dice Scores berechnet – vermutlich passen die Dateimuster nicht. "
                      "Prüfe GT_SUFFIX und schau dir die geloggten Beispiel-Dateinamen an.")
        return

    # CSV
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    with open(output_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Dateiname", "Dice Score"])
        writer.writerows(dice_items)
    logging.info(f"CSV gespeichert: {output_csv}")

    # Statistik & Histogramm
    all_scores = np.array([s for _, s in dice_items], dtype=float)
    n_total = all_scores.size
    n_zeros = int(np.sum(all_scores == 0.0))
    logging.info(f"Anzahl Scores gesamt: {n_total}")
    logging.info(f"Anzahl 0-Scores:      {n_zeros}")
    logging.info(f"Mean inkl. 0:         {np.mean(all_scores):.4f}")

    if EXCLUDE_ZERO_IN_STATS:
        nonzero = all_scores[all_scores > 0]
        if nonzero.size > 0:
            logging.info(f"Mean ohne 0:          {np.mean(nonzero):.4f}")
            save_histogram(
                nonzero,
                title="Dice-Score Verteilung (0-Werte ausgeschlossen)",
                xlabel="Dice Score",
                ylabel="Häufigkeit",
                mean_label="Mean (ohne 0)",
                out_path=output_plot_main
            )
        else:
            logging.warning("Alle Dice-Scores sind 0. Fallback-Histogramm (inkl. 0) wird gespeichert.")
            save_histogram(
                all_scores,
                title="Dice-Score Verteilung (inkl. 0)",
                xlabel="Dice Score",
                ylabel="Häufigkeit",
                mean_label="Mean (inkl. 0)",
                out_path=output_plot_fallback
            )
    else:
        save_histogram(
            all_scores,
            title="Dice-Score Verteilung (inkl. 0)",
            xlabel="Dice Score",
            ylabel="Häufigkeit",
            mean_label="Mean (inkl. 0)",
            out_path=output_plot_fallback
        )

if __name__ == "__main__":
    main()


2025-08-24 11:29:29,336 - INFO - GT-Dateien gefunden: 2838
2025-08-24 11:29:29,337 - INFO - Pred-Dateien gefunden: 5540
2025-08-24 11:29:29,339 - INFO - Beispiel GT-Dateien: atm_001_slice_0_Grund_truth_mask.png, atm_001_slice_10_Grund_truth_mask.png, atm_001_slice_11_Grund_truth_mask.png, atm_001_slice_12_Grund_truth_mask.png, atm_001_slice_13_Grund_truth_mask.png
2025-08-24 11:29:29,340 - INFO - Beispiel Pred-Dateien: atm_001_slice_0_combined.png, atm_001_slice_0_combined_mask.png, atm_001_slice_10_combined.png, atm_001_slice_10_combined_mask.png, atm_001_slice_11_combined.png
2025-08-24 11:29:58,938 - INFO - Verarbeitet: ok=2770, missing_pred=68, load_error=0, empty_gt_skipped=0, exception=0, skip_pattern=0
2025-08-24 11:29:59,185 - INFO - CSV gespeichert: data/upper_airways/combined_segmentations\dice_scores_combined_segmentations_no_zeros.csv
2025-08-24 11:29:59,190 - INFO - Anzahl Scores gesamt: 2770
2025-08-24 11:29:59,193 - INFO - Anzahl 0-Scores:      1517
2025-08-24 11:29:59,1

## Finaler Overlay-Code mit separatem Ausgabeordner

In [3]:
import nibabel as nib

# Zielordner für Overlays
overlay_folder = os.path.join(prediction_folder, "combined_segmentations_overlays")
os.makedirs(overlay_folder, exist_ok=True)

# Overlays für alle berechneten Dice Scores
for filename, score in dice_scores:
    base = filename.replace("_Grund_truth_mask.png", "")
    gt_path = os.path.join(ground_truth_folder, filename)
    pred_path = os.path.join(prediction_folder, f"{base}_combined_mask.png")

    gt = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)
    pred = cv2.imread(pred_path, cv2.IMREAD_GRAYSCALE)

    if gt is None or pred is None:
        continue

    gt_bin = (gt > 0).astype(np.uint8)
    pred_bin = (pred > 0).astype(np.uint8)

    # RGB Overlay erstellen
    overlay = np.zeros((*gt.shape, 3), dtype=np.uint8)
    overlay[gt_bin == 1] = [0, 255, 0]       # Grün: Ground Truth
    overlay[pred_bin == 1] = [255, 0, 0]     # Rot: Prediction
    overlay[(gt_bin == 1) & (pred_bin == 1)] = [255, 255, 0]  # Gelb: Überlappung

    # Original-Slice extrahieren (wenn vorhanden)
    original_image_name = base.split("_slice_")[0] + ".nii.gz"
    slice_idx = int(base.split("_slice_")[1])
    original_image_path = os.path.join("data", "upper_airways", "filtered_images", original_image_name)

    try:
        nii = nib.load(original_image_path)
        original_slice = nii.get_fdata()[:, :, slice_idx]
        original_slice = cv2.normalize(original_slice, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    except Exception as e:
        logging.warning(f"Kein Originalbild geladen für {base}: {e}")
        original_slice = np.zeros(gt.shape, dtype=np.uint8)

    # Darstellung mit Originalbild + Overlay
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(original_slice, cmap="gray")
    axs[0].set_title("Original Slice")
    axs[0].axis("off")
    axs[1].imshow(original_slice, cmap="gray")
    axs[1].imshow(overlay, alpha=0.6)
    axs[1].set_title(f"Overlay\nDice Score: {score:.4f}")
    axs[1].axis("off")

    output_path = os.path.join(overlay_folder, f"{base}_overlay.png")
    plt.savefig(output_path, bbox_inches="tight")
    plt.close()


In [2]:
'''
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import gc
import logging
from tqdm import tqdm
from totalsegmentator.python_api import totalsegmentator

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Ordnerpfade
images_dir = "data/upper_airways/filtered_images"

#images_dir = "data/upper_airways/images"

visualization_dir = "data/upper_airways/segmentation_total_segmentator_upper_airway_results_trachea"
mask_dir = visualization_dir
os.makedirs(visualization_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# Zielgröße
target_size = (1024, 1024)

# Relevante Strukturen
upper_airway_labels = ["trachea"]

# Segmentierung + Speicherung
def segment_and_save(images_dir, visualization_dir, mask_dir):
    image_files = sorted(os.listdir(images_dir))
    temp_output_base = "data/upper_airways/temp_segmentations_total_trachea"
    os.makedirs(temp_output_base, exist_ok=True)

    for image_file in tqdm(image_files, desc="Processing files"):
        image_path = os.path.join(images_dir, image_file)
        if not (image_file.endswith(".nii") or image_file.endswith(".nii.gz")):
            continue

        base_name = os.path.splitext(os.path.splitext(image_file)[0])[0]
        temp_output = os.path.join(temp_output_base, base_name)
        os.makedirs(temp_output, exist_ok=True)

        try:
            totalsegmentator(input=image_path, output=temp_output, task="total")
        except Exception as e:
            logging.error(f"Fehler bei Segmentierung von {image_file}: {e}")
            continue

        nii_img = nib.load(image_path)
        image_data = nii_img.get_fdata()
        affine = nii_img.affine

        total_slices = image_data.shape[2]
        combined_mask = np.zeros_like(image_data, dtype=np.uint8)

        for label in upper_airway_labels:
            seg_path = os.path.join(temp_output, "segmentations", f"{label}.nii.gz")
            if not os.path.exists(seg_path):
                logging.warning(f"{label} nicht gefunden für {base_name}")
                continue

            seg_data = nib.load(seg_path).get_fdata()
            if np.sum(seg_data) == 0:
                logging.info(f"{label} leer in {base_name}")
                continue

            combined_mask = np.logical_or(combined_mask, seg_data > 0)

        if not np.any(combined_mask):
            logging.warning(f"Keine der Zielstrukturen in {base_name} gefunden.")
            continue

        combined_mask = combined_mask.astype(np.uint8)

        for slice_idx in range(total_slices):
            img_slice = image_data[:, :, slice_idx]
            mask_slice = combined_mask[:, :, slice_idx]

            if np.sum(mask_slice) == 0:
                continue

            # Visualisierung
            fig, ax = plt.subplots(1, 2, figsize=(12, 6))
            ax[0].imshow(img_slice, cmap="gray")
            ax[0].set_title(f"Original Slice {slice_idx}")
            ax[0].axis('off')
            ax[1].imshow(img_slice, cmap="gray")
            ax[1].imshow(mask_slice, cmap="jet", alpha=0.5)
            ax[1].set_title(f"Segmentiert Slice {slice_idx}")
            ax[1].axis('off')

            vis_save_path = os.path.join(visualization_dir, f"{base_name}_slice_{slice_idx}_visualization.png")
            plt.savefig(vis_save_path, bbox_inches="tight")
            plt.close(fig)

            # Skaliere Maske
            mask_resized = cv2.resize(mask_slice, target_size, interpolation=cv2.INTER_NEAREST)
            mask_save_path = os.path.join(mask_dir, f"{base_name}_slice_{slice_idx}_segmentation.png")
            cv2.imwrite(mask_save_path, mask_resized * 255)

            gc.collect()

# Ausführen
segment_and_save(images_dir, visualization_dir, mask_dir)
'''

'\nimport os\nimport numpy as np\nimport nibabel as nib\nimport matplotlib.pyplot as plt\nimport cv2\nimport gc\nimport logging\nfrom tqdm import tqdm\nfrom totalsegmentator.python_api import totalsegmentator\n\n# Logging konfigurieren\nlogging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")\n\n# Ordnerpfade\nimages_dir = "data/upper_airways/filtered_images"\n\n#images_dir = "data/upper_airways/images"\n\nvisualization_dir = "data/upper_airways/segmentation_total_segmentator_upper_airway_results_trachea"\nmask_dir = visualization_dir\nos.makedirs(visualization_dir, exist_ok=True)\nos.makedirs(mask_dir, exist_ok=True)\n\n# Zielgröße\ntarget_size = (1024, 1024)\n\n# Relevante Strukturen\nupper_airway_labels = ["trachea"]\n\n# Segmentierung + Speicherung\ndef segment_and_save(images_dir, visualization_dir, mask_dir):\n    image_files = sorted(os.listdir(images_dir))\n    temp_output_base = "data/upper_airways/temp_segmentations_total_trachea"\n    os.ma

In [1]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import gc
import logging
from tqdm import tqdm
from totalsegmentator.python_api import totalsegmentator

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Ordnerpfade
images_dir = "data/upper_airways/filtered_images"
visualization_dir = "data/upper_airways/segmentation_total_segmentator_larynx_air"
mask_dir = "data/upper_airways/segmentation_total_segmentator_larynx_air"
os.makedirs(visualization_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# Zielgröße der Ground-Truth-Bilder
target_size = (1024, 1024)  # (Breite, Höhe)

# Segmentierung + Maske + Visualisierung
def segment_and_save(images_dir, visualization_dir, mask_dir, organ_label="larynx_air"):
    image_files = sorted(os.listdir(images_dir))

    temp_output_base = "data/upper_airways/temp_segmentations_larynx_air"
    os.makedirs(temp_output_base, exist_ok=True)

    for image_file in tqdm(image_files, desc="Processing files"):
        image_path = os.path.join(images_dir, image_file)

        if not (image_file.endswith(".nii") or image_file.endswith(".nii.gz")):
            continue

        # Ordner für diese Datei im temp_output anlegen
        base_name = os.path.splitext(os.path.splitext(image_file)[0])[0]  # z.B. la_018
        temp_output = os.path.join(temp_output_base, base_name)
        if not os.path.exists(temp_output):
            os.makedirs(temp_output)

        try:
            # Segmentierung starten mit speziellem Task
            totalsegmentator(
                input=image_path,
                output=temp_output,
                task="headneck_bones_vessels"
                # roi_subset=[organ_label]
            )
        except Exception as e:
            logging.error(f"Fehler bei der Segmentierung von {image_file}: {e}")
            continue

        # Bilddaten laden
        nii_img = nib.load(image_path)
        image_data = nii_img.get_fdata()

        # Pfad zur Segmentierung im Unterordner
        seg_path = os.path.join(temp_output, f"{organ_label}.nii.gz")
        if not os.path.exists(seg_path):
            logging.warning(f"Keine Segmentierung für {image_file} gefunden.")
            continue

        nii_seg = nib.load(seg_path)
        seg_data = nii_seg.get_fdata()

        total_slices = image_data.shape[2]

        for slice_idx in range(total_slices):
            img_slice = image_data[:, :, slice_idx]
            seg_slice = seg_data[:, :, slice_idx]

            # Binäre Maske
            seg_mask = (seg_slice > 0).astype(np.uint8)

            # Visualisierung
            fig, ax = plt.subplots(1, 2, figsize=(12, 6))
            ax[0].imshow(img_slice, cmap="gray")
            ax[0].set_title(f"Original Slice {slice_idx}")
            ax[0].axis('off')
            ax[1].imshow(img_slice, cmap="gray")
            ax[1].imshow(seg_mask, cmap="jet", alpha=0.5)
            ax[1].set_title(f"Segmentierung Slice {slice_idx}")
            ax[1].axis('off')

            vis_save_path = os.path.join(visualization_dir, f"{base_name}_slice_{slice_idx}_visualization.png")
            plt.savefig(vis_save_path, bbox_inches="tight")
            plt.close(fig)

            # Maske auf 1024 x 1024 skalieren
            seg_mask_resized = cv2.resize(seg_mask, target_size, interpolation=cv2.INTER_NEAREST)

            # Maske speichern
            mask_save_path = os.path.join(mask_dir, f"{base_name}_slice_{slice_idx}_segmentation.png")
            cv2.imwrite(mask_save_path, seg_mask_resized * 255)

            gc.collect()

        # Optional: Temp Ordner komplett löschen
        # Achtung: Wenn du die Ergebnisse behalten willst, bitte nicht löschen!
        # import shutil
        # shutil.rmtree(temp_output)

# Ausführen
segment_and_save(images_dir, visualization_dir, mask_dir)


Processing files:   0%|                                                                         | 0/32 [00:00<?, ?it/s]

No GPU detected. Running on CPU. This can be very slow. The '--fast' or the `--roi_subset` option can help to reduce runtime.

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024




Downloading:   0%|                                                                          | 0.00/231M [00:00<?, ?B/s]
Downloading:   1%|▎                                                                | 1.18M/231M [00:00<00:28, 8.17MB/s]
Downloading:   1%|▌                                                                | 2.10M/231M [00:00<00:26, 8.58MB/s]
Downloading:   1%|▊                                                                | 3.01M/231M [00:00<00:27, 8.32MB/s]
Downloading:   2%|█                                                                | 3.93M/231M [00:00<00:26, 8.54MB/s]
Downloading:   2%|█▍                                                               | 5.11M/231M [00:00<00:24, 9.19MB/s]
Downloading:   3%|█▊                                                               | 6.29M/231M [00:00<00:22, 9.93MB/s]
Downloading:   3%|██▏                                                              | 7.73M/231M [00:00<00:20, 10.8MB/s]
Downloading:   4%|██▌                  

Download finished. Extracting...
Generating rough segmentation for cropping...
Resampling...
  Resampled in 0.42s
Predicting...


F:\Rawan_Masterarbeit\venv_totalseg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.39s/it]


...

No GPU detected. Running on CPU. This can be very slow. The '--fast' or the `--roi_subset` option can help to reduce runtime.

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough segmentation for cropping...
Resampling...
  Resampled in 0.68s
Predicting...



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.70s/it]


  Predicted in 40.70s
Resampling...
  cropping from (512, 512, 58) to (263, 250, 42)
Resampling...
  Resampled in 1.88s
Predicting...



Processing files:  78%|███████████████████████████████████████████████▋             | 25/32 [2:14:11<37:34, 322.08s/it]


KeyboardInterrupt: 

nasal_cavity

pharynx

larynx

trachea

evtl. obere Lungenstrukturen wie lung_upper_lobe_left & lung_upper_lobe_right – aber keine bronchus, lungs, etc.

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import gc
import logging
from tqdm import tqdm
from totalsegmentator.python_api import totalsegmentator

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Ordnerpfade
images_dir = "data/upper_airways/filtered_images"
visualization_dir = "data/upper_airways/segmentation_total_segmentator_trachea_lung"
mask_dir = "data/upper_airways/segmentation_total_segmentator_trachea_lung"
os.makedirs(visualization_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# Zielgröße der Ground-Truth-Bilder
target_size = (1024, 1024)  # (Breite, Höhe)

# Segmentierung + Maske + Visualisierung
def segment_and_save(images_dir, visualization_dir, mask_dir, organ_label=["trachea", "lung_upper_lobe_left", "lung_upper_lobe_right"]):
    image_files = sorted(os.listdir(images_dir))
    temp_output_base = "data/upper_airways/temp_segmentations_trachea_lung"
    os.makedirs(temp_output_base, exist_ok=True)

    for image_file in tqdm(image_files, desc="Processing files"):
        if not (image_file.endswith(".nii") or image_file.endswith(".nii.gz")):
            continue

        image_path = os.path.join(images_dir, image_file)
        base_name = os.path.splitext(os.path.splitext(image_file)[0])[0]
        temp_output = os.path.join(temp_output_base, base_name)
        os.makedirs(temp_output, exist_ok=True)

        try:
            totalsegmentator(
                input=image_path,
                output=temp_output,
                task="total"
            )
        except Exception as e:
            logging.error(f"Fehler bei der Segmentierung von {image_file}: {e}")
            continue

        try:
            nii_img = nib.load(image_path)
            image_data = nii_img.get_fdata()
        except Exception as e:
            logging.error(f"Fehler beim Laden von {image_file}: {e}")
            continue

        total_slices = image_data.shape[2]

        for organ in organ_label:
            seg_path = os.path.join(temp_output, f"{organ}.nii.gz")
            if not os.path.exists(seg_path):
                logging.warning(f"Keine Segmentierung für {organ} in {image_file} gefunden.")
                continue

            try:
                nii_seg = nib.load(seg_path)
                seg_data = nii_seg.get_fdata()
            except Exception as e:
                logging.error(f"Fehler beim Laden der Segmentierung {organ} für {image_file}: {e}")
                continue

            for slice_idx in range(total_slices):
                img_slice = image_data[:, :, slice_idx]
                seg_slice = seg_data[:, :, slice_idx]
                seg_mask = (seg_slice > 0).astype(np.uint8)

                # Visualisierung
                fig, ax = plt.subplots(1, 2, figsize=(12, 6))
                ax[0].imshow(img_slice, cmap="gray")
                ax[0].set_title(f"Original Slice {slice_idx}")
                ax[0].axis('off')
                ax[1].imshow(img_slice, cmap="gray")
                ax[1].imshow(seg_mask, cmap="jet", alpha=0.5)
                ax[1].set_title(f"{organ} Slice {slice_idx}")
                ax[1].axis('off')

                vis_save_path = os.path.join(visualization_dir, f"{base_name}_{organ}_slice_{slice_idx}_visualization.png")
                plt.savefig(vis_save_path, bbox_inches="tight")
                plt.close(fig)

                # Maske speichern
                seg_mask_resized = cv2.resize(seg_mask, target_size, interpolation=cv2.INTER_NEAREST)
                mask_save_path = os.path.join(mask_dir, f"{base_name}_{organ}_slice_{slice_idx}_segmentation.png")
                cv2.imwrite(mask_save_path, seg_mask_resized * 255)

                gc.collect()

# Funktion ausführen
segment_and_save(images_dir, visualization_dir, mask_dir)


In [2]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import gc
import logging
from tqdm import tqdm
from totalsegmentator.python_api import totalsegmentator

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Ordnerpfade
images_dir = "data/upper_airways/filtered_images"
visualization_dir = "data/upper_airways/segmentation_total_segmentator_nasel"
mask_dir = "data/upper_airways/segmentation_total_segmentator_nasel"
os.makedirs(visualization_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# Zielgröße der Ground-Truth-Bilder
target_size = (1024, 1024)  # (Breite, Höhe)

# Segmentierung + Maske + Visualisierung
def segment_and_save(images_dir, visualization_dir, mask_dir, organ_label=["nasal_cavity_right", "nasal_cavity_left"]):
    image_files = sorted(os.listdir(images_dir))
    temp_output_base = "data/upper_airways/temp_segmentations_nasel"
    os.makedirs(temp_output_base, exist_ok=True)

    for image_file in tqdm(image_files, desc="Processing files"):
        if not (image_file.endswith(".nii") or image_file.endswith(".nii.gz")):
            continue

        image_path = os.path.join(images_dir, image_file)
        base_name = os.path.splitext(os.path.splitext(image_file)[0])[0]
        temp_output = os.path.join(temp_output_base, base_name)
        os.makedirs(temp_output, exist_ok=True)

        try:
            totalsegmentator(
                input=image_path,
                output=temp_output,
                task="head_glands_cavities"
            )
        except Exception as e:
            logging.error(f"Fehler bei der Segmentierung von {image_file}: {e}")
            continue

        try:
            nii_img = nib.load(image_path)
            image_data = nii_img.get_fdata()
        except Exception as e:
            logging.error(f"Fehler beim Laden von {image_file}: {e}")
            continue

        total_slices = image_data.shape[2]

        for organ in organ_label:
            seg_path = os.path.join(temp_output, f"{organ}.nii.gz")
            if not os.path.exists(seg_path):
                logging.warning(f"Keine Segmentierung für {organ} in {image_file} gefunden.")
                continue

            try:
                nii_seg = nib.load(seg_path)
                seg_data = nii_seg.get_fdata()
            except Exception as e:
                logging.error(f"Fehler beim Laden der Segmentierung {organ} für {image_file}: {e}")
                continue

            for slice_idx in range(total_slices):
                img_slice = image_data[:, :, slice_idx]
                seg_slice = seg_data[:, :, slice_idx]
                seg_mask = (seg_slice > 0).astype(np.uint8)

                # Visualisierung
                fig, ax = plt.subplots(1, 2, figsize=(12, 6))
                ax[0].imshow(img_slice, cmap="gray")
                ax[0].set_title(f"Original Slice {slice_idx}")
                ax[0].axis('off')
                ax[1].imshow(img_slice, cmap="gray")
                ax[1].imshow(seg_mask, cmap="jet", alpha=0.5)
                ax[1].set_title(f"{organ} Slice {slice_idx}")
                ax[1].axis('off')

                vis_save_path = os.path.join(visualization_dir, f"{base_name}_{organ}_slice_{slice_idx}_visualization.png")
                plt.savefig(vis_save_path, bbox_inches="tight")
                plt.close(fig)

                # Maske speichern
                seg_mask_resized = cv2.resize(seg_mask, target_size, interpolation=cv2.INTER_NEAREST)
                mask_save_path = os.path.join(mask_dir, f"{base_name}_{organ}_slice_{slice_idx}_segmentation.png")
                cv2.imwrite(mask_save_path, seg_mask_resized * 255)

                gc.collect()

# Funktion ausführen
segment_and_save(images_dir, visualization_dir, mask_dir)


Processing files:   0%|                                                                         | 0/32 [00:00<?, ?it/s]

No GPU detected. Running on CPU. This can be very slow. The '--fast' or the `--roi_subset` option can help to reduce runtime.

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough segmentation for cropping...
Resampling...
  Resampled in 0.40s
Predicting...


F:\Rawan_Masterarbeit\venv_totalseg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


  Predicted in 21.90s
Resampling...
  cropping from (512, 512, 32) to (407, 388, 32)
Resampling...
  Resampled in 2.28s
Predicting...



100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:34<00:00, 11.85s/it]


...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.61s/it]


  Predicted in 32.77s
Resampling...
  cropping from (512, 512, 69) to (470, 367, 45)
Resampling...
  Resampled in 4.50s
Predicting...



100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [03:33<00:00, 11.87s/it]


  Predicted in 234.04s
Resampling...
Saving segmentations...
  Saved in 49.20s


Processing files: 100%|█████████████████████████████████████████████████████████████| 32/32 [6:24:04<00:00, 720.13s/it]


In [1]:
import os
import nibabel as nib
import numpy as np

segmentation_folder = r"F:\Rawan_Masterarbeit\data\upper_airways\temp_segmentations_nasel\atm_001"

files = os.listdir(segmentation_folder)
print(f"Gefundene Dateien: {files}")

empty_files = []
non_empty_files = []

for file in files:
    if file.endswith(".nii") or file.endswith(".nii.gz"):
        file_path = os.path.join(segmentation_folder, file)
        print(f"Lade Datei: {file_path}")
        img = nib.load(file_path)
        data = img.get_fdata()

        max_val = np.max(data)
        print(f"Max Wert in Datei {file}: {max_val}")

        if max_val == 0:
            empty_files.append(file)
        else:
            non_empty_files.append(file)

print("\n❌ Leere Dateien (kein Segment vorhanden):")
for f in empty_files:
    print(f)

print(f"\n✅ Nicht-leere Dateien: {len(non_empty_files)}")
for f in non_empty_files:
    print(f)


Gefundene Dateien: ['eye_left.nii.gz', 'optic_nerve_left.nii.gz', 'optic_nerve_right.nii.gz', 'eye_right.nii.gz', 'eye_lens_right.nii.gz', 'eye_lens_left.nii.gz', 'parotid_gland_left.nii.gz', 'submandibular_gland_right.nii.gz', 'parotid_gland_right.nii.gz', 'nasopharynx.nii.gz', 'submandibular_gland_left.nii.gz', 'oropharynx.nii.gz', 'nasal_cavity_right.nii.gz', 'hypopharynx.nii.gz', 'nasal_cavity_left.nii.gz', 'auditory_canal_right.nii.gz', 'auditory_canal_left.nii.gz', 'soft_palate.nii.gz', 'hard_palate.nii.gz']
Lade Datei: F:\Rawan_Masterarbeit\data\upper_airways\temp_segmentations_nasel\atm_001\eye_left.nii.gz
Max Wert in Datei eye_left.nii.gz: 1.0
Lade Datei: F:\Rawan_Masterarbeit\data\upper_airways\temp_segmentations_nasel\atm_001\optic_nerve_left.nii.gz
Max Wert in Datei optic_nerve_left.nii.gz: 0.0
Lade Datei: F:\Rawan_Masterarbeit\data\upper_airways\temp_segmentations_nasel\atm_001\optic_nerve_right.nii.gz
Max Wert in Datei optic_nerve_right.nii.gz: 0.0
Lade Datei: F:\Rawan_Ma

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import gc
import logging
from tqdm import tqdm
from totalsegmentator.python_api import totalsegmentator

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Ordnerpfade
images_dir = "data/upper_airways/filtered_images"
visualization_dir = "data/upper_airways/segmentation_total_segmentator_nasel_pharynx"
mask_dir = "data/upper_airways/segmentation_total_segmentator_nasel_pharynx"
os.makedirs(visualization_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# Zielgröße der Ground-Truth-Bilder
target_size = (1024, 1024)  # (Breite, Höhe)

# Segmentierung + Maske + Visualisierung
def segment_and_save(images_dir, visualization_dir, mask_dir, organ_label=["nasal_cavity_right", 
                                                                           "nasal_cavity_left",
                                                                           "nasopharynx",
                                                                           "oropharynx", 
                                                                           "hypopharynx"]):
    image_files = sorted(os.listdir(images_dir))
    temp_output_base = "data/upper_airways/temp_segmentations_nasel_pharynx"
    os.makedirs(temp_output_base, exist_ok=True)

    for image_file in tqdm(image_files, desc="Processing files"):
        if not (image_file.endswith(".nii") or image_file.endswith(".nii.gz")):
            continue

        image_path = os.path.join(images_dir, image_file)
        base_name = os.path.splitext(os.path.splitext(image_file)[0])[0]
        temp_output = os.path.join(temp_output_base, base_name)
        os.makedirs(temp_output, exist_ok=True)

        try:
            totalsegmentator(
                input=image_path,
                output=temp_output,
                task="head_glands_cavities"
            )
        except Exception as e:
            logging.error(f"Fehler bei der Segmentierung von {image_file}: {e}")
            continue

        try:
            nii_img = nib.load(image_path)
            image_data = nii_img.get_fdata()
        except Exception as e:
            logging.error(f"Fehler beim Laden von {image_file}: {e}")
            continue

        total_slices = image_data.shape[2]

        for organ in organ_label:
            seg_path = os.path.join(temp_output, f"{organ}.nii.gz")
            if not os.path.exists(seg_path):
                logging.warning(f"Keine Segmentierung für {organ} in {image_file} gefunden.")
                continue

            try:
                nii_seg = nib.load(seg_path)
                seg_data = nii_seg.get_fdata()
            except Exception as e:
                logging.error(f"Fehler beim Laden der Segmentierung {organ} für {image_file}: {e}")
                continue

            for slice_idx in range(total_slices):
                img_slice = image_data[:, :, slice_idx]
                seg_slice = seg_data[:, :, slice_idx]
                seg_mask = (seg_slice > 0).astype(np.uint8)

                # Visualisierung
                fig, ax = plt.subplots(1, 2, figsize=(12, 6))
                ax[0].imshow(img_slice, cmap="gray")
                ax[0].set_title(f"Original Slice {slice_idx}")
                ax[0].axis('off')
                ax[1].imshow(img_slice, cmap="gray")
                ax[1].imshow(seg_mask, cmap="jet", alpha=0.5)
                ax[1].set_title(f"{organ} Slice {slice_idx}")
                ax[1].axis('off')

                vis_save_path = os.path.join(visualization_dir, f"{base_name}_{organ}_slice_{slice_idx}_visualization.png")
                plt.savefig(vis_save_path, bbox_inches="tight")
                plt.close(fig)

                # Maske speichern
                seg_mask_resized = cv2.resize(seg_mask, target_size, interpolation=cv2.INTER_NEAREST)
                mask_save_path = os.path.join(mask_dir, f"{base_name}_{organ}_slice_{slice_idx}_segmentation.png")
                cv2.imwrite(mask_save_path, seg_mask_resized * 255)

                gc.collect()

# Funktion ausführen
segment_and_save(images_dir, visualization_dir, mask_dir)


Processing files:   0%|                                                                         | 0/32 [00:00<?, ?it/s]

No GPU detected. Running on CPU. This can be very slow. The '--fast' or the `--roi_subset` option can help to reduce runtime.

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough segmentation for cropping...
Resampling...
  Resampled in 0.63s
Predicting...


F:\Rawan_Masterarbeit\venv_totalseg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.50s/it]


  Predicted in 36.78s
Resampling...
  cropping from (512, 512, 32) to (407, 388, 32)
Resampling...
  Resampled in 3.33s
Predicting...



100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [03:49<00:00, 28.74s/it]


...

Processing files:  62%|███████████████████████████████████▋                     | 20/32 [20:03:42<4:02:51, 1214.26s/it]

No GPU detected. Running on CPU. This can be very slow. The '--fast' or the `--roi_subset` option can help to reduce runtime.

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough segmentation for cropping...
Resampling...
  Resampled in 0.66s
Predicting...



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.91s/it]


  Predicted in 43.84s
Resampling...
  cropping from (512, 512, 60) to (455, 358, 46)
Resampling...
  Resampled in 4.50s
Predicting...



 74%|████████████████████████████████████████████████████████████▋                     | 20/27 [04:00<01:23, 12.00s/it]

In [ ]:
# probiere mit dem ganz orginalen daten.